In [ ]:
# Метрики:
# RPU - rub per uet - стоимость 1 УЕТы
# FPV - filling per visit - отношение числа пломб к числу приемов
# DPV - devitalization per vizit - отношение числа мышьяков к числу приемов
# Плановое среднее значение стоимости 1 УЕТы в 2021 году = 166,26 руб.
# Плановое среднее значение стоимости 1 УЕТы в 2022 году = 167,70 руб.

In [ ]:
# PRVS:
# 72 - хирурги
# 87 - физио

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import pyodbc
from simpledbf import Dbf5
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from plotly.colors import n_colors

from datetime import date, datetime
from pandas.tseries.offsets import MonthEnd, MonthBegin

import hashlib
import json

from openpyxl import load_workbook
from openpyxl import worksheet

%matplotlib inline

In [ ]:
# Определим максимальный период, уже находящийся в директории для загрузки

def get_max_loaded_period(cur_path):
    
    max_period = 0
    max_loaded_period = ''
    
    for dr in os.listdir(cur_path):
        if int(dr[-5:-1]) > max_period:
            max_period = int(dr[-5:-1])
            max_loaded_period = dr
            
    return max_loaded_period

In [ ]:
# Рекурсивный обход директории и поиск u-файлов, возврат списка u-файлов

def read_u_files(cur_path, dbf_files):
    
    for dr in os.listdir(cur_path):
        
        abs_path = os.path.join(cur_path, dr)

        if os.path.isdir(abs_path):
            read_u_files(abs_path, dbf_files)
        elif 'u' in dr[0].lower() and dr[-4:].lower() == '.dbf':
            dbf_files.append(abs_path)

    return dbf_files

In [ ]:
# Рекурсивный обход директории и поиск d-файлов, возврат списка d-файлов

def read_d_files(cur_path, dbf_files):
    
    for dr in os.listdir(cur_path):
        
        abs_path = os.path.join(cur_path, dr)
        
        if os.path.isdir(abs_path):
            read_d_files(abs_path, dbf_files)
        elif 'd' in dr[0].lower() and dr[-4:].lower() == '.dbf':
            dbf_files.append(abs_path)

    return dbf_files

In [ ]:
# Рекурсивный обход директории и поиск z-файлов, возврат списка z-файлов

def read_z_files(cur_path, dbf_files):
    
    for dr in os.listdir(cur_path):
        
        abs_path = os.path.join(cur_path, dr)
        
        if os.path.isdir(abs_path):
            read_z_files(abs_path, dbf_files)
        elif 'z' in dr[0].lower() and dr[-4:].lower() == '.dbf':
            dbf_files.append(abs_path)

    return dbf_files

In [ ]:
# Загрузим u-файлы, выберем только нужные колонки
def read_u_dbf(year=0):
#     Передаем путь, по которому найдем все dbf файлы для загрузки  
    if year == 0:  
        u_files_to_load = read_u_files(dbf_path_all, [])
    else:
        u_files_to_load = read_u_files(dbf_path_year, [])
    df_u = pd.DataFrame()
    for f in u_files_to_load:
        # Сразу забираем только нужные колонки
        df_tmp = Dbf5(f, codec='cp866').to_dataframe()
        df_tmp = df_tmp[['OT_PER', 'MSK_OT', 'PERSCODE', 'NHISTORY', 'PROFIL', 'MKB1', 'MKB2', 'MKB3', 'CODE_USL',
                         'CODE_MD', 'DATE_IN', 'KOL_USL', 'RES_GOSP', 'ISH_MOV', 'TARIF_B', 'SUM_RUB', 'CODE_OTD',
                         'P_CEL', 'MKB0', 'C_ZAB', 'OT_PER_U', 'IDCASE']]
        df_u = pd.concat([df_u, df_tmp], ignore_index=True)
        # Этот вариант на 50% дольше
        # df_u = pd.concat([df_u, Dbf5(f, codec='cp866').to_dataframe()], ignore_index=True)
        # break
    return df_u

In [ ]:
# 0 - загрузим все года
def read_d_dbf(year=0):
#     Передаем путь, по которому найдем все dbf файлы для загрузки
    if year == 0:
        d_files_to_load = read_d_files(dbf_path_all, [])
    else:
        d_files_to_load = read_d_files(dbf_path_year, [])
    df_d = pd.DataFrame()
    for f in d_files_to_load:
        df_tmp = Dbf5(f, codec='cp866').to_dataframe()
        df_tmp = df_tmp[['OT_PER', 'CODE_MD', 'FIO_MD', 'KATEG_MD', 'SPEC_MD', 'MD_SS', 'PRVS']]
        df_tmp['MSK_OT'] = f[-10:-8]  # Добавим код страховой для верного объединения с u-файлом
        df_d = pd.concat([df_d, df_tmp], ignore_index=True)
    return df_d

In [ ]:
# Загрузим z-файлы, выберем только нужные колонки
def read_z_dbf(year=0):
#     Передаем путь, по которому найдем все dbf файлы для загрузки  
    if year == 0:  
        z_files_to_load = read_z_files(dbf_path_all, [])
    else:
        z_files_to_load = read_z_files(dbf_path_year, [])
    df_z = pd.DataFrame()
    for f in z_files_to_load:
        # Сразу забираем только нужные колонки
        df_tmp = Dbf5(f, codec='cp866').to_dataframe()
        df_tmp = df_tmp[['OT_PER', 'PERSCODE', 'NHISTORY', 'CODE_USL','DATE_OUT', 'TEETH_CODE']]
        df_z = pd.concat([df_z, df_tmp], ignore_index=True)
        
    df_z = df_z.rename(columns={'DATE_OUT': 'DATE_IN'})
    return df_z

In [ ]:
# year - загрузим только нужный год
# year = 0 - загрузим все года
year = 2022

# структура папок должна соответствовать (.../dbf/year/.../*.dbf, например .../dbf/2022/.../*.dbf)
dbf_path_all = '../no_mkb2_u0621/dbf'
dbf_path_year = f'../no_mkb2_u0621/dbf/{year}'

In [ ]:
# Перезапишем максимальный период (этот период периодически пересобирается)
# Предыдущие периоды не трогаем, если вдруг нужно их перезаписать, то копируем руками выборочно нужный период
# Новый период появляется уже в новом месяце, когда предыдущий период уже собран полностью и не требует перезаписи

# Создадим временную заглушку для year == 0, нужно будет продумать загрузку всех периодов и нужно ли?
if year == 0:
    year = 2022

reestr_path = 'D:/Электронная регистратура/BIN_MIS/bin_mo/Reestr/TFOMS_1/'

max_loaded_period = get_max_loaded_period(dbf_path_year)

shutil.copytree(src=os.path.join(reestr_path, max_loaded_period), 
                dst=f'{dbf_path_year}/{max_loaded_period}', 
                dirs_exist_ok=True,
                ignore = shutil.ignore_patterns('*.zip', '*.xls', '*.enc', '*.pdf'))

# Смотрим, появился ли в каталоге реестров новый период, больше чем уже загруженный
# Директории имеют название "REESTR_3613015005221", где:
# REESTR_+{код=361301}+{регион=50}+{период_ММГГ=0522}+{номер_пакета=1}
# Тогда добавляя 1000 к численной части названия директории мы по сути будем итерироваться по месяцам
next_period = f'REESTR_{int(max_loaded_period[-13:]) + 1000}'
next_period_path = os.path.join(reestr_path, next_period)

# Мало ли мы не запускали скрипт несколько месяцев, тогда в цикле подгрузим все директории,
# более поздние относительно уже загруженной
while os.path.isdir(os.path.join(reestr_path, next_period)):
    shutil.copytree(src=next_period_path, 
                    dst=f'{dbf_path_year}/{next_period}', 
                    dirs_exist_ok=True,
                    ignore = shutil.ignore_patterns('*.zip', '*.xls', '*.enc', '*.pdf'))
    next_period = f'REESTR_{int(next_period[-13:]) + 1000}'
    next_period_path = os.path.join(reestr_path, next_period)

In [ ]:
# Получим актуальный максимальный период в исходных загружаемых данных
current_max_period = get_max_loaded_period(dbf_path_year)[-5:-1]
current_max_period

In [ ]:
df_all_u = read_u_dbf(year)
df_all_d = read_d_dbf(year)
df_all_z = read_z_dbf(year)

In [ ]:
# Плановая средняя стоимость 1 УЕТ
# Стоимости УЕТ разнятся в зависимости от типа услуги

mean_uet = 0
if year == 2021:
    mean_uet = 166.26
elif year == 2022:
    mean_uet = 167.7

In [ ]:
# # В OT_PER_U == OT_PER попадают перевыставленные случаи, а значит присутсвуют в прошлых периодах
# df_all_u = df_all_u[df_all_u['OT_PER'] == df_all_u['OT_PER_U']]

# df_all_u['DATE_IN'] = pd.to_datetime(df_all_u['DATE_IN'])

# # Загрузим тарифы в УЕТ (берем октябрь 2021, так как в течение 2021 года тарифы в УЕТ не изменялись
# # для верности будем брать mdu посвежее, хотя изменений по стоматологии там и не было
# # и объеденим с u-датафреймом
# # df_mdu = pd.read_excel('../no_mkb2_u0621/dbf/mdu1021_dbf.xlsx')
# df_mdu = pd.read_excel('../no_mkb2_u0621/dbf/mdu0122_dbf.xlsx')
# df_mdu = df_mdu[df_mdu['KOL_UET'] > 0]
# df_all_u_uet = pd.merge(df_all_u, df_mdu[['CODE_USL', 'NAME_USL', 'KOL_UET']], how='left', on='CODE_USL')
# df_all = pd.merge(df_all_u_uet, df_all_d[['OT_PER', 'MSK_OT', 'CODE_MD', 'FIO_MD', 'SPEC_MD', 'MD_SS', 'PRVS']],
#                   how='left', on=['OT_PER', 'MSK_OT', 'CODE_MD'])
# df_all['SUM_UET'] = df_all['KOL_UET']*df_all['KOL_USL']

# df_all['SUM_RUB'] = np.around(df_all['SUM_RUB'], decimals=2)
# # Отметим физио отдельно
# df_all.loc[df_all['PRVS'] == 87, 'FIO_MD'] += ' физио'

# # В 2021 была корректировка ФИО
# df_all['FIO_MD'] = df_all['FIO_MD'].str.replace(pat='Горбикова Элла Рашитовна',
#                                                 repl='Горбикова Эльфия Рашитовна')
# df_all['FIO_MD'] = df_all['FIO_MD'].str.replace(pat='Камынова Наталия Валерьевна',
#                                                 repl='Камынова Наталия Валериевна')

In [ ]:
# В OT_PER_U == OT_PER попадают перевыставленные случаи, а значит присутсвуют в прошлых периодах
df_all_u = df_all_u[df_all_u['OT_PER'] == df_all_u['OT_PER_U']]

df_all_u['DATE_IN'] = pd.to_datetime(df_all_u['DATE_IN'])
df_all_z['DATE_IN'] = pd.to_datetime(df_all_z['DATE_IN'])

# Загрузим тарифы в УЕТ (берем октябрь 2021, так как в течение 2021 года тарифы в УЕТ не изменялись
# для верности будем брать mdu посвежее, хотя изменений по стоматологии там и не было
# и объеденим с u-датафреймом
# df_mdu = pd.read_excel('../no_mkb2_u0621/dbf/mdu1021_dbf.xlsx')
df_mdu = pd.read_excel('../no_mkb2_u0621/dbf/mdu0122_dbf.xlsx')
df_mdu = df_mdu[df_mdu['KOL_UET'] > 0]
df_all = df_all_u \
            .merge(df_mdu[['CODE_USL', 'NAME_USL', 'KOL_UET']], how='left', on='CODE_USL') \
            .merge(df_all_d[['OT_PER', 'MSK_OT', 'CODE_MD', 'FIO_MD', 'SPEC_MD', 'MD_SS', 'PRVS']],
                   how='left', on=['OT_PER', 'MSK_OT', 'CODE_MD']) \
#             .merge(df_all_z, how='left', on=['OT_PER', 'PERSCODE', 'NHISTORY', 'CODE_USL', 'DATE_IN'])

df_all['SUM_UET'] = df_all['KOL_UET']*df_all['KOL_USL']

df_all['SUM_RUB'] = np.around(df_all['SUM_RUB'], decimals=2)
# Отметим физио отдельно
df_all.loc[df_all['PRVS'] == 87, 'FIO_MD'] += ' физио'

# В 2021 была корректировка ФИО
df_all['FIO_MD'] = df_all['FIO_MD'].str.replace(pat='Горбикова Элла Рашитовна',
                                                repl='Горбикова Эльфия Рашитовна')
df_all['FIO_MD'] = df_all['FIO_MD'].str.replace(pat='Камынова Наталия Валерьевна',
                                                repl='Камынова Наталия Валериевна')

In [ ]:
# Обозначение типа специальности

df_all['spec'] = df_all \
                    .PRVS \
                    .map({208: 'тер', 72: 'хир', 69: 'тер', 71: 'тер', 87: 'физио'})

In [ ]:
# Этот вариант компактный, но что если будет врач без ФИО, плюс физио (добавка после отчества) не выделяется

# fio_map = {x: x.split()[0][0:5] + x.split()[1][0:1] + x.split()[2][0:1] for x in df_all.FIO_MD.unique()}
# fio_map

In [ ]:
# Добавим сокращенное ФИО для выгрузки в дашборд

fio_map = {}
for fio in df_all.FIO_MD.unique():
    short_fio = ''
    f = True  # У первого элемента (фамилии) нужно оставить 5 символов, у остальных по 1
    for part_fio in fio.split():
        if f:
            short_fio += part_fio[0:3]
            f = False  # Все остальные уже не первые
        else:
            short_fio += part_fio[0:1]
    fio_map.update({fio: short_fio})
fio_map

df_all['fio_short'] = df_all \
                        .FIO_MD \
                        .map(fio_map)

In [ ]:
# Запись Excel выгрузок со статистиками по средним значениям УЕТ и ТАП

# 1. total_usl - анализ услуг по их стоимости и отклолнению от среднего значения 1 УЕТ
df_to_excel = df_all.groupby(['CODE_USL', 'NAME_USL'], as_index=False) \
    .agg({'KOL_USL': 'sum', 'SUM_RUB': 'sum', 'SUM_UET': 'sum', 'KOL_UET': 'max', 'TARIF_B': 'max'})

# DEVIATION - отклонение в рублях по услуге от средней плановой стоимости 1 УЕТы
df_to_excel['DEVIATION'] = df_to_excel['SUM_RUB']-df_to_excel['KOL_USL']*mean_uet*df_to_excel['KOL_UET']

# Доля услуги в общем доходе и выработке
total_rub = df_to_excel['SUM_RUB'].sum()
total_uet = df_to_excel['SUM_UET'].sum()
df_to_excel['part_rub'] = np.around(df_to_excel['SUM_RUB']*100/total_rub, decimals=2)
df_to_excel['part_uet'] = np.around(df_to_excel['SUM_UET']*100/total_uet, decimals=2)
df_to_excel['part_diff'] = df_to_excel['part_rub'] - df_to_excel['part_uet']
df_to_excel.to_excel('total_usl.xlsx')

# 2. total_usl_by_doc - аналогично п.1, но в разбивке по врачу (позволит найти существенные проблемы)
df_to_excel = df_all.groupby(['FIO_MD', 'CODE_USL', 'NAME_USL'], as_index=False).\
    agg({'KOL_USL': 'sum', 'SUM_RUB': 'sum', 'SUM_UET': 'sum', 'KOL_UET': 'max', 'TARIF_B': 'max'})
df_to_excel['DEVIATION'] = df_to_excel['SUM_RUB']-df_to_excel['KOL_USL']*mean_uet*df_to_excel['KOL_UET']
df_to_excel['part_rub'] = np.around(df_to_excel['SUM_RUB']*100/total_rub, decimals=2)
df_to_excel['part_uet'] = np.around(df_to_excel['SUM_UET']*100/total_uet, decimals=2)
df_to_excel['part_diff'] = df_to_excel['part_rub'] - df_to_excel['part_uet']
df_to_excel.to_excel('total_usl_by_doc.xlsx')

# 3. total_usl_by_doc_period - аналогично п.2, но еще и в разбивке по месяцам
df_mean_tap = df_all.groupby(['OT_PER', 'FIO_MD', 'NHISTORY'], as_index=False) \
    .agg({'SUM_UET': 'sum'}) \
    .groupby(['OT_PER', 'FIO_MD'], as_index=False) \
    .agg({'SUM_UET': 'mean'}) \
    .rename(columns={'SUM_UET': 'MEAN_TAP'})
df_to_excel = df_all.groupby(['OT_PER', 'FIO_MD'], as_index=False) \
    .agg({'SUM_RUB': 'sum', 'SUM_UET': 'sum'})
df_to_excel = pd.merge(df_to_excel, df_mean_tap, how='left', on=['OT_PER', 'FIO_MD'])
df_to_excel['DEVIATION'] = df_to_excel['SUM_RUB']-df_to_excel['SUM_UET']*mean_uet
df_to_excel = df_to_excel.pivot(index='FIO_MD', columns='OT_PER', values=['DEVIATION', 'MEAN_TAP'])

# По каждой колонке добавим строку с итогами
columns = [column for column in df_to_excel.columns]
data = []
for column in columns:
    data.append(df_to_excel[column].sum())
itog = pd.DataFrame(data=[data], columns=columns, index=['ИТОГО:'])
df_to_excel = pd.concat([df_to_excel, itog])
df_to_excel.to_excel("total_usl_by_doc_period.xlsx", float_format='%.1f', index_label='ФИО врача')
wb = load_workbook("total_usl_by_doc_period.xlsx")
ws = wb.active

# Установим ширину столбца с ФИО
ws.column_dimensions['A'] = worksheet.dimensions.ColumnDimension(ws, index='A', width=38)
wb.save("total_usl_by_doc_period.xlsx")

# 4. total_usl_cheapest_by_doc - 10 самых "дешевых" услуг по каждлому врачу

df_to_excel = df_all.groupby(['FIO_MD', 'CODE_USL', 'NAME_USL'], as_index=False).\
    agg({'KOL_USL': 'sum', 'SUM_RUB': 'sum', 'SUM_UET': 'sum', 'KOL_UET': 'max', 'TARIF_B': 'max'})
df_to_excel['DEVIATION'] = df_to_excel['SUM_RUB']-df_to_excel['KOL_USL']*mean_uet*df_to_excel['KOL_UET']
df_to_excel['part_rub'] = np.around(df_to_excel['SUM_RUB']*100/total_rub, decimals=2)
df_to_excel['part_uet'] = np.around(df_to_excel['SUM_UET']*100/total_uet, decimals=2)
df_to_excel['part_diff'] = df_to_excel['part_rub'] - df_to_excel['part_uet']
df_to_xls = df_to_excel.sort_values(['FIO_MD', 'DEVIATION']).groupby('FIO_MD', as_index=False).head(10)
df_to_xls \
    .rename(columns={'FIO_MD': 'ФИО', 'CODE_USL': 'Код', 'NAME_USL': 'Услуга', 'KOL_USL': 'Кол-во',
                     'SUM_RUB': 'Всего Руб', 'SUM_UET': 'Всего УЕТ', 'KOL_UET': 'Тариф УЕТ', 'TARIF_B': 'Тариф Руб',
                     'DEVIATION': 'Отклонение', 'part_rub': 'Доля в руб', 'part_uet': 'Доля в УЕТ',
                     'part_diff': 'Доля отклонения'}) \
    .to_excel('total_usl_cheapest_by_doc.xlsx')

# 5. total_usl_expensive_by_doc - 10 самых "дешевых" услуг по каждлому врачу

df_to_excel = df_all.groupby(['FIO_MD', 'CODE_USL', 'NAME_USL'], as_index=False) \
    .agg({'KOL_USL': 'sum', 'SUM_RUB': 'sum', 'SUM_UET': 'sum', 'KOL_UET': 'max', 'TARIF_B': 'max'})
df_to_excel['DEVIATION'] = df_to_excel['SUM_RUB']-df_to_excel['KOL_USL']*mean_uet*df_to_excel['KOL_UET']
df_to_excel['part_rub'] = np.around(df_to_excel['SUM_RUB']*100/total_rub, decimals=2)
df_to_excel['part_uet'] = np.around(df_to_excel['SUM_UET']*100/total_uet, decimals=2)
df_to_excel['part_diff'] = df_to_excel['part_rub'] - df_to_excel['part_uet']
df_to_xls = df_to_excel \
    .sort_values(['FIO_MD', 'DEVIATION'], ascending=['True', 'False']) \
    .groupby('FIO_MD', as_index=False) \
    .tail(10)
df_to_xls \
    .rename(columns={'FIO_MD': 'ФИО', 'CODE_USL': 'Код', 'NAME_USL': 'Услуга', 'KOL_USL': 'Кол-во',
                     'SUM_RUB': 'Всего Руб', 'SUM_UET': 'Всего УЕТ', 'KOL_UET': 'Тариф УЕТ', 'TARIF_B': 'Тариф Руб',
                     'DEVIATION': 'Отклонение', 'part_rub': 'Доля в руб', 'part_uet': 'Доля в УЕТ',
                     'part_diff': 'Доля отклонения'}) \
    .to_excel('total_usl_expensive_by_doc.xlsx')

In [ ]:
# Услуги приема

priem = ['B01.065.007V', 'B01.065.008V', 'B01.065.001V', 'B01.065.002V', 'B01.065.003V', 'B01.065.004V', \
         'B01.067.001V', 'B01.067.002V', 'B04.065.006V', 'B04.065.002V', 'B04.065.004V', 'B01.065.005V', \
         'B01.065.006V']

In [ ]:
# Услуги первичного приема

priem_first = ['B01.065.007V', 'B01.065.001V', 'B01.065.003V', 'B01.067.001V', 'В01.003.001V', 'B01.065.005V']

In [ ]:
# Услуги установки пломбы

filling = ['A16.07.002.001V', 'A16.07.002.002V', 'A16.07.002.003V', 'A16.07.002.004V', 'A16.07.002.005V', \
           'A16.07.002.006V', 'A16.07.002.007V', 'A16.07.002.008V', 'A16.07.002.010V', 'A16.07.002.011V', \
           'A16.07.002.012V']

In [ ]:
# Услуга установки мышьяка

devit = 'A11.07.027V'

In [ ]:
# Услуга стоматоскопии

stomatoskop = 'A03.07.001V'

In [ ]:
# Приемы по врачу

priem_by_doc = df_all \
                .query('CODE_USL in @priem') \
                .groupby('FIO_MD') \
                .agg({'KOL_USL': 'count'}) \
                .rename(columns={'KOL_USL': 'preim'})

In [ ]:
# Приемы по врачу и месяцу

priem_by_doc_month = df_all \
                        .query('CODE_USL in @priem') \
                        .groupby(['FIO_MD', 'OT_PER_U']) \
                        .agg({'KOL_USL': 'count'}) \
                        .rename(columns={'KOL_USL': 'preim'}) 

In [ ]:
# Первичные приемы и стоматоскопии по врачу и месяцу

stomatoskop_by_doc_month = df_all \
                            .query('CODE_USL in @priem_first') \
                            .groupby(['FIO_MD', 'OT_PER_U']) \
                            .agg({'KOL_USL': 'count'}) \
                            .rename(columns={'KOL_USL': 'Первичных приемов'}) \
                            .merge(df_all
                                        .query('CODE_USL in @stomatoskop')
                                        .groupby(['FIO_MD', 'OT_PER_U'])
                                        .agg({'KOL_USL': 'count'})
                                        .rename(columns={'KOL_USL': 'Люмин. стоматоскопий'}), 
                                   left_index=True, 
                                   right_index=True, 
                                   how='left') \
                            .reset_index() \
                            .pivot(index='FIO_MD', columns='OT_PER_U', values=['Первичных приемов', 'Люмин. стоматоскопий']) \
                            .fillna(0) \
                            .astype('int')
stomatoskop_by_doc_month \
    .to_excel(f'Стоматоскопии_{year}.xlsx')

In [ ]:
# Пломбы по врачу

filling_by_doc = df_all \
                    .query('CODE_USL in @filling') \
                    .groupby('FIO_MD') \
                    .agg({'KOL_USL': 'sum'}) \
                    .rename(columns={'KOL_USL': 'filling'})

In [ ]:
# Пломбы по врачу и месяцу

filling_by_doc_month = df_all \
                        .query('CODE_USL in @filling') \
                        .groupby(['FIO_MD', 'OT_PER_U']) \
                        .agg({'KOL_USL': 'sum'}) \
                        .rename(columns={'KOL_USL': 'filling'})

In [ ]:
# Наложение девит пасты по врачу

devit_by_doc = df_all \
                    .query('CODE_USL == @devit') \
                    .groupby('FIO_MD') \
                    .agg({'KOL_USL': 'sum'}) \
                    .rename(columns={'KOL_USL': 'devit'})

In [ ]:
# Наложение девит пасты по врачу и месяцу

devit_by_doc_month = df_all \
                        .query('CODE_USL == @devit') \
                        .groupby(['FIO_MD', 'OT_PER_U']) \
                        .agg({'KOL_USL': 'sum'}) \
                        .rename(columns={'KOL_USL': 'devit'})

In [ ]:
# Диагнозы (основной диагноз из ТАП) по врачу

diag_by_doc = df_all \
                .groupby(['FIO_MD', 'MKB1', 'NHISTORY'], as_index=False) \
                .agg({'NHISTORY': 'count'}) \
                .rename(columns={'NHISTORY': 'diag_count'}) \
                .groupby(['FIO_MD', 'MKB1'], as_index=False) \
                .agg({'diag_count': 'count'}) \
                .sort_values(['FIO_MD', 'diag_count'], ascending=[True, False])

In [ ]:
# Диагнозы (основной диагноз из ТАП) по врачу и месяцу

diag_by_doc_month = df_all \
                        .groupby(['FIO_MD', 'MKB1', 'NHISTORY', 'OT_PER_U'], as_index=False) \
                        .agg({'NHISTORY': 'count'}) \
                        .rename(columns={'NHISTORY': 'diag_count'}) \
                        .groupby(['FIO_MD', 'MKB1', 'OT_PER_U'], as_index=False) \
                        .agg({'diag_count': 'count'}) \
                        .sort_values(['FIO_MD', 'diag_count'], ascending=[True, False])

In [ ]:
# Самые частые диагнозы

top_mkb = diag_by_doc \
                .groupby('MKB1', as_index=False) \
                .agg({'diag_count': 'sum'}) \
                .sort_values('diag_count', ascending=False) \
                .head(8) \
                .MKB1.tolist()

In [ ]:
# Выборка по списку самых частых диагнозов в разрезе врачей

top_diag_by_doc = diag_by_doc \
                    .query('MKB1 in @top_mkb') \
                    .pivot(index='FIO_MD', columns='MKB1', values='diag_count') \
                    .fillna(0) \
                    .astype('int')

In [ ]:
# Выборка по списку самых частых диагнозов в разрезе врачей и месяцев

top_diag_by_doc_month = diag_by_doc_month \
                            .query('MKB1 in @top_mkb') \
                            .pivot(index=['FIO_MD', 'OT_PER_U'], columns='MKB1', values='diag_count') \
                            .fillna(0) \
                            .astype('int')

In [ ]:
# Средняя стоимость ТАП в УЕТ

avg_uet_by_doc = df_all \
                    .groupby(['FIO_MD', 'NHISTORY'], as_index=False) \
                    .agg({'SUM_UET': 'sum'}) \
                    .groupby('FIO_MD') \
                    .agg({'SUM_UET': 'mean'}) \
                    .round(2) \
                    .rename(columns={'SUM_UET': 'avg_uet'})

In [ ]:
# Средняя стоимость ТАП в УЕТ по месяцам

avg_uet_by_doc_month = df_all \
                        .groupby(['FIO_MD', 'NHISTORY', 'OT_PER_U'], as_index=False) \
                        .agg({'SUM_UET': 'sum'}) \
                        .groupby(['FIO_MD', 'OT_PER_U']) \
                        .agg({'SUM_UET': 'mean'}) \
                        .round(2) \
                        .rename(columns={'SUM_UET': 'avg_uet'})

In [ ]:
# Средняя стоимость ТАП в рублях

avg_rub_by_doc = df_all \
                    .groupby(['FIO_MD', 'NHISTORY'], as_index=False) \
                    .agg({'SUM_RUB': 'sum'}) \
                    .groupby('FIO_MD') \
                    .agg({'SUM_RUB': 'mean'}) \
                    .round() \
                    .astype(int) \
                    .rename(columns={'SUM_RUB': 'avg_rub'})

In [ ]:
# Средняя стоимость ТАП в рублях по месяцам

avg_rub_by_doc_month = df_all \
                        .groupby(['FIO_MD', 'NHISTORY', 'OT_PER_U'], as_index=False) \
                        .agg({'SUM_RUB': 'sum'}) \
                        .groupby(['FIO_MD', 'OT_PER_U']) \
                        .agg({'SUM_RUB': 'mean'}) \
                        .round() \
                        .astype(int) \
                        .rename(columns={'SUM_RUB': 'avg_rub'})

In [ ]:
# Средняя стоимость УЕТ
# RPU - rub per uet - стоимость 1 УЕТы

avg_uet_rub = df_all \
                    .groupby('FIO_MD', as_index=False) \
                    .agg({'SUM_RUB': 'sum', 'SUM_UET': 'sum'})
avg_uet_rub['RPU'] = round(avg_uet_rub['SUM_RUB'] / avg_uet_rub['SUM_UET'], 1)

In [ ]:
# Средняя стоимость УЕТ по месяцам
# RPU - rub per uet - стоимость 1 УЕТы

avg_uet_rub_month = df_all \
                        .groupby(['FIO_MD', 'OT_PER_U'], as_index=False) \
                        .agg({'SUM_RUB': 'sum', 'SUM_UET': 'sum'})
avg_uet_rub_month['RPU'] = round(avg_uet_rub_month['SUM_RUB'] / avg_uet_rub_month['SUM_UET'], 1)

In [ ]:
# По-хорошему написать функцию получения рабочего/нерабочего дня и часла рабочих дней в году
# Пока такой бэйзлайн

# Создадим список нерабочих дней и рабочих суббот

holidays = ['2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07',
            '2022-01-07', '2022-02-23', '2022-03-07', '2022-03-08', '2022-05-02',
            '2022-05-03', '2022-05-09', '2022-05-10', '2022-06-13', '2022-11-04']
working_saturday = ['2022-03-05']

In [ ]:
# Рабочие дни по месяцам в 2022 году

working_days_by_month = {1: 16, 2: 19, 3: 22, 4: 21, 5: 18, 6: 21,
                         7: 21, 8: 23, 9: 22, 10: 21, 11: 21, 12: 22}

In [ ]:
# Здесь будет максимальная дата, загруженная из dbf

with open('last_date.json') as f:
    config = json.load(f)

last_dbf_date = config['last_date']

In [ ]:
# Зашифруем IDCASE, выгрузим csv для даша

df_to_dash = df_all[['OT_PER', 'CODE_USL', 'DATE_IN', 'KOL_USL', 'TARIF_B', 'SUM_RUB', 'MSK_OT',
                     'CODE_OTD', 'KOL_UET', 'FIO_MD', 'fio_short', 'SUM_UET', 'spec', 'IDCASE']]
df_to_dash['TEETH_CODE'] = np.nan
df_to_dash['id'] = df_to_dash['IDCASE'].apply(lambda x: hashlib.sha1(str(x).encode()).hexdigest())

# В текущем месяце мы не грузим последний незавершенный день
# Если вдруг у нас максимальная дата в данных больше, чем сегодня, то мы это увидим в заголовке дашборда
# А потеря ошибочно отсеченного день будет практически незаметна

# Когда автоматизируем загрузку данных до начала рабочего дня, то эту проверку надо будет убрать
# Если загружаем текущий месяц, то последний день надо удрать, так как данные за последний день не полные
if df_to_dash['DATE_IN'].max().month == pd.Timestamp.today().month:
    df_to_dash = df_to_dash[df_to_dash['DATE_IN'] < df_to_dash['DATE_IN'].max()]

# Отметим рабочий или нерабочий день
df_to_dash['is_business_day'] = df_to_dash['DATE_IN'].isin(working_saturday) | \
                                        (~(df_to_dash['DATE_IN'].isin(holidays)) & 
                                         ~(df_to_dash['DATE_IN'].dt.dayofweek >= 5))

# Отметим дежурные субботы
df_to_dash['is_saturday'] = ((df_to_dash['DATE_IN'].dt.dayofweek == 5) & ~df_to_dash['DATE_IN'].isin(working_saturday))
    
df_to_dash \
    .drop(columns=['FIO_MD', 'IDCASE']) \
    .to_csv('dash_data.csv', sep=';', index=False)    

df_to_dash \
    .query('DATE_IN <= @last_dbf_date') \
    .drop(columns=['fio_short', 'id']) \
    .to_csv('dash_data_superset.csv', sep=';', index=False)

In [ ]:
df_to_dash.groupby('DATE_IN').agg({'SUM_UET': 'sum'})

In [ ]:
# Проверим какое предсказание выполнения плана лучше
# monthly - относительно среднего выполнения в день за текущий месяц
# totally - относительно среднего выполнения в день за весь период

df_check_prediction = df_to_dash.query('MSK_OT != "50"')
df_check_prediction['month'] = df_check_prediction['DATE_IN'].dt.month
df_check_prediction['plan_bdays'] = df_check_prediction \
                                        .OT_PER.str[0:2] \
                                        .apply(int) \
                                        .map(working_days_by_month)

In [ ]:
df_check = df_check_prediction.groupby('DATE_IN').agg({'SUM_UET': 'sum',
                                                       'is_business_day': 'min',
                                                       'plan_bdays': 'min'})
df_check

In [ ]:
# Поэтому пробежим по датафрейму и в словари будем записывать накопительные суммы по месяцу
# Переписать df.groupby(df.index.month)['n'].cumsum()

month = df_check.index[0].month
s = 0
d = 0
max_uet = 0
rolling_sum = {}
rolling_days = {}
uet_fact = {}
uet_monthly_fact = {}

for index, row in df_check.iterrows():
    if month == index.month:
        s += row['SUM_UET']
        rolling_sum[index] = s
        
        d += row['is_business_day']
        rolling_days[index] = d
        
        if s > max_uet:
            max_uet = s
        uet_fact[month] = s
        
    else:
        s = 0
        s += row['SUM_UET']
        rolling_sum[index] = s
        
        d = 0
        d += row['is_business_day']
        rolling_days[index] = d
        
        month = index.month
        max_uet = s
        uet_fact[month] = s
        
for index, row in df_check.iterrows():
    uet_monthly_fact[index] = uet_fact[index.month]

In [ ]:
rol_sum = pd.Series(data=rolling_sum, name='rol_sum_monthly')
rol_days = pd.Series(data=rolling_days, name='rol_days_monthly')
monthly_fact_uet = pd.Series(data=uet_monthly_fact, name='monthly_fact_uet')

In [ ]:
df_check['cum_sum_uet'] = df_check.SUM_UET.cumsum()
df_check['cum_sum_days'] = df_check.is_business_day.cumsum()

In [ ]:
df_check = df_check \
            .merge(rol_sum, left_index=True, right_index=True) \
            .merge(rol_days, left_index=True, right_index=True) \
            .merge(monthly_fact_uet, left_index=True, right_index=True) \
            .query('is_business_day > 0')

In [ ]:
df_check['avg_monthly'] = df_check['rol_sum_monthly'] / df_check['rol_days_monthly']
df_check['avg_total'] = df_check['cum_sum_uet'] / df_check['cum_sum_days']

In [ ]:
df_check['plan_uet'] = 34546

In [ ]:
df_check['predict_monthly'] = df_check['rol_sum_monthly'] + \
                                (df_check['plan_bdays'] - df_check['rol_days_monthly']) * df_check['avg_monthly']
df_check['predict_totally'] = df_check['rol_sum_monthly'] + \
                                (df_check['plan_bdays'] - df_check['rol_days_monthly']) * df_check['avg_total']

In [ ]:
df_check

In [ ]:
# Прогноз на основании среднедневного выполнения за текущий месяц более точное для конкретного набора данных

px.line(data_frame=df_check[['predict_monthly', 'predict_totally', 'monthly_fact_uet']])

In [ ]:
sns.lineplot(data=df_check[['plan_uet', 'predict_monthly', 'predict_totally', 'rol_sum_monthly']])

In [ ]:
df_check_prediction.set_index('DATE_IN', drop=False)

In [ ]:
df_to_dash.dtypes

In [ ]:
df_to_dash \
    .query('DATE_IN >= "2022-06-01"') \
    .groupby('DATE_IN') \
    .agg({'SUM_UET': 'sum'}).mean()

In [ ]:
df_to_dash

In [ ]:
sns.distplot(df_all.groupby(['FIO_MD', 'NHISTORY']).agg({'SUM_UET': 'sum'}), kde=False)

In [ ]:
sns.distplot(np.log(df_all.groupby(['FIO_MD', 'NHISTORY']).agg({'SUM_UET': 'sum'})), kde=False)

In [ ]:
# Объединим все в итоговую таблицу, в качестве базы возьмем уникальных врачей из исходных данных

In [ ]:
df_result = df_all.groupby('FIO_MD').agg({'NHISTORY': 'nunique'}).rename(columns={'NHISTORY': 'tap_count'})
df_result = df_result \
                .merge(priem_by_doc, how='left', left_index=True, right_index=True) \
                .merge(filling_by_doc, how='left', left_index=True, right_index=True) \
                .merge(devit_by_doc, how='left', left_index=True, right_index=True) \
                .merge(top_diag_by_doc, how='left', left_index=True, right_index=True) \
                .merge(avg_rub_by_doc, how='left', left_index=True, right_index=True) \
                .fillna(0) \
                .astype(int) \
                .merge(avg_uet_by_doc, how='left', left_index=True, right_index=True) \
                .drop(columns='tap_count') \
                .reset_index() \
                .merge(avg_uet_rub[['FIO_MD', 'RPU']], on='FIO_MD') \
                .rename(columns={'FIO_MD': 'Врач', 'preim': 'Приемов', 'filling': 'Пломб', 'devit': 'Мышьяков'})

In [ ]:
df_result_by_month = df_all \
                        .groupby(['FIO_MD', 'OT_PER_U']) \
                        .agg({'NHISTORY': 'nunique'}) \
                        .rename(columns={'NHISTORY': 'tap_count'}) \
                        .merge(priem_by_doc_month, how='left', left_index=True, right_index=True) \
                        .merge(filling_by_doc_month, how='left', left_index=True, right_index=True) \
                        .merge(devit_by_doc_month, how='left', left_index=True, right_index=True) \
                        .merge(top_diag_by_doc_month, how='left', left_index=True, right_index=True) \
                        .merge(avg_rub_by_doc_month, how='left', left_index=True, right_index=True) \
                        .fillna(0) \
                        .astype(int) \
                        .merge(avg_uet_by_doc_month, how='left', left_index=True, right_index=True) \
                        .drop(columns='tap_count') \
                        .reset_index() \
                        .merge(avg_uet_rub_month[['FIO_MD', 'OT_PER_U', 'RPU']], on=['FIO_MD', 'OT_PER_U']) \
                        .rename(columns={'FIO_MD': 'Врач', 'preim': 'Приемов', 'filling': 'Пломб', 
                                         'devit': 'Мышьяков', 'OT_PER_U': 'Месяц'})                        

In [ ]:
# FPV - filling per visit - отношение числа пломб к числу приемов
# DPV - devitalization per vizit - отношение числа мышьяков к числу приемов

df_result['FPV'] = df_result['Пломб'] / df_result['Приемов']
df_result['DPV'] = df_result['Мышьяков'] / df_result['Приемов']
df_result['DPV %'] = df_result['DPV'] * 100

In [ ]:
# Метрики мо месяцам
# FPV - filling per visit - отношение числа пломб к числу приемов
# DPV - devitalization per vizit - отношение числа мышьяков к числу приемов

df_result_by_month['FPV'] = df_result_by_month['Пломб'] / df_result_by_month['Приемов']
df_result_by_month['DPV'] = df_result_by_month['Мышьяков'] / df_result_by_month['Приемов']
df_result_by_month['DPV %'] = df_result_by_month['DPV'] * 100

In [ ]:
# AD_HOC запрос

df_all \
    .query('spec == "тер" & OT_PER != "0622" & MSK_OT != "50" & MSK_OT != "99"') \
    .groupby(['FIO_MD', 'OT_PER'], as_index=False) \
    .agg({'SUM_UET': 'sum'}) \
    .groupby('FIO_MD') \
    .describe() \
    .round(1) \
    .to_excel('doc_stat.xlsx')

In [ ]:
df_all \
    .query('OT_PER != "0622" & MSK_OT != "50" & MSK_OT != "99"') \
    .groupby(['spec', 'OT_PER']) \
    .agg({'NHISTORY': 'nunique'}) \
    .sum()

In [ ]:
df_all \
    .query('OT_PER != "0622" & MSK_OT != "50" & MSK_OT != "99"') \
    .groupby(['spec', 'OT_PER'], as_index=False) \
    .agg({'NHISTORY': 'nunique'}) \
    .groupby('spec') \
    .describe() \
    .round(1) \
    .to_excel('visit_stat.xlsx')

In [ ]:
df_result.to_excel(f'Лечение_{year}.xlsx', index=False)
df_result_by_month.to_excel(f'Лечение_по_месяцам_{year}.xlsx', index=False)

In [ ]:
diag_by_doc.groupby('MKB1', as_index=False) \
    .agg({'diag_count': 'sum'}) \
    .sort_values('diag_count', ascending=False) \
    .head(10)

In [ ]:
# Число пломб на приеме по врачу (средний FPV за год)

px.bar(df_result.query('(FPV > 0) & (Врач != "Коваленко Ольга Эдуардовна физио")').sort_values('FPV', ascending=False), 
       x='Врач',
       y='FPV',
       title=f"Число пломб на приеме в {year} году")

In [ ]:
# Сделать grayscale

greys = n_colors('rgb(100, 100, 100)', 'rgb(255, 255, 255)', len(df_result_by_month['Месяц'].unique())+1, colortype='rgb')

In [ ]:
# Средний FPV по месяцам и врачу (ч/б)

px.bar(df_result_by_month.query('(FPV > 0) & (Врач != "Коваленко Ольга Эдуардовна физио")'), 
       x='Врач',
       y='FPV',
       color='Месяц',
       barmode='group',
       color_discrete_sequence=greys,
       title=f"Число пломб на приеме в {year} году").update_xaxes(categoryorder='total descending') \
    .update_layout(plot_bgcolor='rgb(255,255,255)') \
    .update_yaxes(gridcolor='grey')

In [ ]:
# Средний FPV по месяцам и врачу (цвет)

px.bar(df_result_by_month.query('(FPV > 0) & (Врач != "Коваленко Ольга Эдуардовна физио")'), 
       x='Врач',
       y='FPV',
       color='Месяц',
       barmode='group',
       title=f"Число пломб на приеме в {year} году").update_xaxes(categoryorder='total descending')

In [ ]:
# Средний DPV за год по врачу

px.bar(df_result.query('DPV > 0').sort_values('DPV', ascending=False), 
       x='Врач',
       y='DPV %',
       title=f"Процент приемов с мышьяком в {year} году")

In [ ]:
# DPV по месяцу и врачу (ч/б)

px.bar(df_result_by_month.query('DPV > 0'), 
       x='Врач',
       y='DPV %',
       color='Месяц',
       barmode='group',
       color_discrete_sequence=greys,
       title=f"Процент приемов с мышьяком в {year} году").update_xaxes(categoryorder='total descending') \
    .update_layout(plot_bgcolor='rgb(255,255,255)') \
    .update_yaxes(gridcolor='grey')

In [ ]:
# DPV по месяцу и врачу (цвет)

px.bar(df_result_by_month.query('DPV > 0'), 
       x='Врач',
       y='DPV %',
       color='Месяц',
       barmode='group',
       title=f"Процент приемов с мышьяком в {year} году").update_xaxes(categoryorder='total descending')

In [ ]:
# Средняя стоимость ТАП в руб по врачу

px.bar(df_result.sort_values('avg_rub', ascending=False), 
       x='Врач',
       y='avg_rub',
       title=f'Средняя стоимость ТАП в руб. за {year} году') \
    .update_yaxes(title_text = 'Руб')

In [ ]:
# Средняя стоимость ТАП в руб по врачу и месяцу (ч/б), вертикальные бары

px.bar(df_result_by_month, 
       x='Врач',
       y='avg_rub',
       color='Месяц',
       barmode='group',
       width=1060,
       color_discrete_sequence=greys,
       title=f'Средняя стоимость ТАП в руб. за {year} году') \
    .update_layout(plot_bgcolor='rgb(255,255,255)') \
    .update_yaxes(title_text = 'Руб', gridcolor='grey') \
    .update_xaxes(categoryorder='total descending')

In [ ]:
# Средняя стоимость ТАП в руб по врачу и месяцу (цв), горизонтальные бары

px.bar(df_result_by_month, 
       y='Врач',
       x='avg_rub',
       color='Месяц',
       barmode='group',
#        width=1060,
       height = 2000,
       orientation = 'h',
       title=f'Средняя стоимость ТАП в руб. за {year} году') \
    .update_yaxes(title_text = 'Руб', categoryorder='total ascending') 

In [ ]:
# Средняя стоимость ТАП в УЕТ по врачу

px.bar(df_result.sort_values('avg_uet', ascending=False), 
       x='Врач',
       y='avg_uet',
       title=f'Средняя стоимость ТАП в УЕТ за {year} год') \
    .update_yaxes(title_text = 'УЕТ')

In [ ]:
# Средняя стоимость ТАП в УЕТ по врачу и месяцу (ч/б)

px.bar(df_result_by_month, 
       x='Врач',
       y='avg_uet',
       color='Месяц',
       barmode='group',
       width=1660,
       color_discrete_sequence=greys,
       title=f'Среднее количество УЕТ в ТАП за {year} год') \
    .update_layout(plot_bgcolor='rgb(255,255,255)') \
    .update_yaxes(title_text = 'УЕТ', gridcolor='grey') \
    .update_xaxes(categoryorder='total descending')

In [ ]:
# Средняя стоимость ТАП в УЕТ по врачу и месяцу (цв)

px.bar(df_result_by_month, 
       x='Врач',
       y='avg_uet',
       color='Месяц',
       barmode='group',
       width=1660,
       title=f'Среднее количество УЕТ в ТАП за {year} год') \
    .update_yaxes(title_text = 'УЕТ') \
    .update_xaxes(categoryorder='total descending')

In [ ]:
# Средняя стоимость 1 УЕТ в руб по врачу

px.bar(df_result.query('Врач != "Коваленко Ольга Эдуардовна физио"').sort_values('RPU', ascending=False), 
                x='Врач',
                y='RPU',
                title=f'Средняя стоимость 1 УЕТ в руб за {year} год (план: {mean_uet})',
                text='RPU',
                width=1360) \
    .update_traces(textfont_size=20, textposition="outside", textangle=-45, cliponaxis=False) \
    .update_layout(plot_bgcolor='rgb(255,255,255)')
# fig.show(renderer="notebook_connected")

In [ ]:
# Средняя стоимость 1 УЕТ в руб по врачу и месяцу (ч/б)

px.bar(df_result_by_month.query('Врач != "Коваленко Ольга Эдуардовна физио"'), 
                x='Врач',
                y='RPU',
                title=f'Средняя стоимость 1 УЕТ в руб за {year} год (план: {mean_uet})',
                text='RPU',
                color='Месяц',
                barmode='group',
                color_discrete_sequence=greys,
                width=1360) \
    .update_layout(plot_bgcolor='rgb(255,255,255)') \
    .update_yaxes(gridcolor='grey', dtick = 167.7) \
    .update_xaxes(categoryorder='total descending') \
#     .update_traces(textfont_size=20, textposition="outside", textangle=-45, cliponaxis=False) \
# fig.show(renderer="notebook_connected")

In [ ]:
# Средняя стоимость 1 УЕТ в руб по врачу и месяцу (цв)

px.bar(df_result_by_month.query('Врач != "Коваленко Ольга Эдуардовна физио"'), 
                x='Врач',
                y='RPU',
                title=f'Средняя стоимость 1 УЕТ в руб за {year} год (план: {mean_uet})',
                text='RPU',
                color='Месяц',
                barmode='group',
                width=1360) \
    .update_yaxes(gridcolor='black', dtick = 167.7) \
    .update_xaxes(categoryorder='total descending')

In [ ]:
# Ниже work in progress

In [ ]:
# Рабочих дней в текущем месяце

wdays_passed = df_to_dash[(df_to_dash['DATE_IN'].dt.month == df_to_dash['DATE_IN'].max().month) &
                          (df_to_dash['DATE_IN'].dt.dayofweek < 5)] \
                    .query('DATE_IN not in @holidays') \
                    .DATE_IN.nunique()

In [ ]:
# Суббот (в данных) в текущем месяце

saturday_passed = df_to_dash[(df_to_dash['DATE_IN'].dt.month == df_to_dash['DATE_IN'].max().month) &
                             (df_to_dash['DATE_IN'].dt.dayofweek == 5)] \
                        .query('DATE_IN not in @working_saturday') \
                        .DATE_IN.nunique()

In [ ]:
# Число рабочих дней в текущем месяце

wdyas_month = working_days_by_month[df_to_dash['DATE_IN'].max().month]

In [ ]:
# Осталось рабочих дней

wdays_remain = wdyas_month - wdays_passed

In [ ]:
# Всего число суббот в текущем месяце

month_end = df_to_dash['DATE_IN'].max() + MonthEnd(1)
month_begin = df_to_dash['DATE_IN'].max() - MonthBegin(1)

# Делаем + 1, т.к. разница month_end - month_begin теряет 1 день
num_weeks, remainder = divmod((month_end - month_begin).days + 1, 7)  
if (5 - month_begin.weekday()) % 7 < remainder:
    saturdays_month = num_weeks + 1
else:
    saturdays_month = num_weeks

In [ ]:
saturdays_remain = saturdays_month - saturday_passed

In [ ]:
# Среднее число и стоимость ТАП с начала года по будням по специальности
# Так как считаем план, то берем без МТР

df_to_dash[(df_to_dash['DATE_IN'].dt.dayofweek < 5) & 
           (df_to_dash['DATE_IN'] > '2022-01-09')] \
    .query('(DATE_IN not in @holidays) & (MSK_OT !="50")') \
    .groupby('spec') \
    .agg({'IDCASE': 'nunique', 'DATE_IN': 'nunique', 'SUM_UET': 'sum'}) \
    .assign(avg_tap        = lambda x: x['IDCASE'] / x['DATE_IN'],
            avg_uet_tap    = lambda x: x['SUM_UET'] / x['IDCASE'],
            uet_per_day    = lambda x: x['avg_tap'] * x['avg_uet_tap'],
            uet_this_month = lambda x: x['uet_per_day'] * wdays_remain).sum()

In [ ]:
# Среднее число и стоимость ТАП с начала года по субботам по специальности (2022-03-05 - рабочая суббота)

df_to_dash[(df_to_dash['DATE_IN'].dt.dayofweek == 5) & 
           (df_to_dash['DATE_IN'] > '2022-01-09')] \
    .query('(DATE_IN not in @working_saturday) & (MSK_OT !="50")') \
    .groupby('spec') \
    .agg({'IDCASE': 'nunique', 'DATE_IN': 'nunique', 'SUM_UET': 'sum'}) \
    .assign(avg_tap        = lambda x: x['IDCASE'] / x['DATE_IN'],
            avg_uet_tap    = lambda x: x['SUM_UET'] / x['IDCASE'],
            uet_per_day    = lambda x: x['avg_tap'] * x['avg_uet_tap'],
            uet_this_month = lambda x: x['uet_per_day'] * saturdays_remain).sum()

In [ ]:
df_to_dash[(df_to_dash['DATE_IN'] >= '2022-05-01')] \
    .query('MSK_OT !="50"') \
    .groupby('spec') \
    .agg({'SUM_UET': 'sum'}).sum()

In [ ]:
# Число рабочих дней по врачам с указанием специальности

working_days = df_all.groupby(['FIO_MD', 'PRVS', 'DATE_IN'], as_index=False) \
                        .agg({'NHISTORY': 'nunique', 'SUM_UET': 'sum', 'SUM_RUB': 'sum'}) \
                        .groupby('FIO_MD') \
                        .agg({'PRVS': 'min', 'DATE_IN': 'count', 'SUM_UET': 'sum', 'SUM_RUB': 'sum', 'NHISTORY': 'mean'})
working_days['spec'] = working_days \
                            .PRVS \
                            .map({208: 'тер', 72: 'хир', 69: 'тер', 71: 'тер', 87: 'физио'})
working_days.drop(columns='PRVS', inplace=True)

In [ ]:
working_days.SUM_RUB.sum()

In [ ]:
working_days.SUM_UET.sum()

In [ ]:
# Среднедневная выработка УЕТ по специальностям

working_days.groupby('spec').agg({'SUM_UET': 'mean'})

In [ ]:
# Статистика по талонам в день

working_days.query('spec == "тер"').NHISTORY.describe()

In [ ]:
# Число рабочих дней в год по специальности (факт)
# План по терапевтам 4047 (19 врачей Скалкович+Кукушкина пока принимают условно по 0.5 приема) по средней в 213 дней)
# План по терапевтам 3990 (19 врачей Скалкович+Кукушкина пока принимают условно по 0.5 приема) по медиане 210 дней)
# План по хирургам 1314 (6 врачей по 219 дней - среднее по факту 2021 года при медиане 223)

working_days.groupby('spec').agg({'DATE_IN': 'sum'})

In [ ]:
# Статистика по числу рабочих дней

working_days.query('spec == "тер"').DATE_IN.describe()

In [ ]:
# Медианное число рабочих дней терапевтов в 2021 (факт) - Медианное число рабочих дней терапевтов в 2022

ter_days = 210 - working_days.query('spec == "тер"').DATE_IN.median()
ter_days

In [ ]:
# Медианное число рабочих дней хирургов в 2021 (факт) - Медианное число рабочих дней хирургов в 2022
# Чистая медиана 223
# Среднее без Саркисова 219

hir_days = 219 - working_days.query('spec == "хир"').DATE_IN.median()
hir_days

In [ ]:
# Плановые расчеты ниже возможно не завершены (основной расчет был в ноутбуке за 2021 год)

# План на 2022 год

plan2022_oms = 414548
plan2022_total = 414548*100/88
plan2022_total

In [ ]:
# План по специальностям на 2022 год

ter_plan = plan2022_total*78.8/100
hir_plan = plan2022_total*17.9/100
fizio_plan = plan2022_total*3.3/100

In [ ]:
# Число врачей по специальностям
ter_count = 19
hir_count = 6

In [ ]:
# План на 2022 год без января

plan2022_corr = plan2022_total - working_days.SUM_UET.sum()

In [ ]:
# План по специальностям на 2022 год без января

ter_plan_corr = plan2022_corr*78.8/100
hir_plan_corr = plan2022_corr*17.9/100
fizio_plan_corr = plan2022_corr*3.3/100

In [ ]:
ter_plan_corr/ter_count/ter_days

In [ ]:
hir_plan_corr/hir_count/hir_days

In [ ]:
fizio_plan_corr/1/253

In [ ]:
# 

plan2022_total/12 - working_days.SUM_UET.sum()

In [ ]:
# Число рабочих дней по врачам с указанием специальности

working_days = df_all.groupby(['FIO_MD', 'PRVS', 'DATE_IN'], as_index=False) \
                        .agg({'NHISTORY': 'nunique', 'SUM_UET': 'sum'}) \
                        .groupby('FIO_MD') \
                        .agg({'PRVS': 'min', 'DATE_IN': 'count', 'SUM_UET': 'mean', 'NHISTORY': 'mean'})
working_days['spec'] = working_days \
                            .PRVS \
                            .map({208: 'тер', 72: 'хир', 69: 'тер', 71: 'тер', 87: 'физио'})
working_days.drop(columns='PRVS', inplace=True)

In [ ]:
# Выполнение УЕТ по дням и количество врачей по специальностям в этот день

uet_by_day = df_all \
                .groupby(['DATE_IN', 'spec'], as_index=False) \
                .agg({'FIO_MD': 'nunique'}) \
                .pivot(index='DATE_IN', columns='spec', values='FIO_MD') \
                .fillna(0) \
                .astype('int32') \
                .merge(df_all \
                        .groupby('DATE_IN') \
                        .agg({'SUM_UET': 'sum'}),
                       on='DATE_IN',
                       how='left'
                      )

In [ ]:
uet_by_day[uet_by_day.index.dayofweek < 5]

In [ ]:
# Среднее УЕТ в день

plt.figure(figsize=(15,10))
sns.lineplot(data=uet_by_day[uet_by_day.index.dayofweek < 5], 
             x='DATE_IN', 
             y=uet_by_day[uet_by_day.index.dayofweek < 5].SUM_UET,
             label='УЕТ')
sns.lineplot(data=uet_by_day[uet_by_day.index.dayofweek < 5], 
             x='DATE_IN', 
             y=(uet_by_day[uet_by_day.index.dayofweek < 5].тер)*100,
             label='Число терапевтов')
sns.lineplot(data=uet_by_day[uet_by_day.index.dayofweek < 5], 
             x='DATE_IN', 
             y=(uet_by_day[uet_by_day.index.dayofweek < 5].хир)*100,
             label='Число хирургов')

In [ ]:
uet_by_day[uet_by_day.index.dayofweek < 5].corr()

In [ ]:
uet_by_day[uet_by_day.index.dayofweek < 5].corr(method='spearman')

In [ ]:
uet_by_day[uet_by_day.index.dayofweek < 5].corr(method='kendall')